# DATA LOADING

In [ ]:
#imports for the notebook
import requests
import os
import json
import time

## Data Collection - During Season

### Set-Up

In [14]:
#FPL API
data_url = "https://fantasy.premierleague.com/api/"

#Create Directory for Data
data_dir = "raw_data"
os.makedirs(data_dir, exist_ok=True)

In [21]:
#Fetch & Save Data
def get_save_data(url, filename, subdir = None):
    
    if subdir:
        full_dir = os.path.join(data_dir, subdir)
        os.makedirs(full_dir, exist_ok = True)
        filepath = os.path.join(full_dir, filename)
    else:
        filepath = os.path.join(data_dir, filename) 

    print(f"Fetching data from: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        with open(filepath, 'w', encoding = "utf-8") as f:
            json.dump(data, f, indent = 5)
        print(f"Data saved to: {filepath}")
        return data
    except Exception as e:
        print(f"Error occurred at {url}: {e}")
        return None
    finally:
        time.sleep(1)

In [ ]:
#Collect Season Data
static_data = get_save_data(f"{data_url}bootstrap-static/", "static_data.json", )

if static_data:
    print("Saved")
else:
    print("Error getting data")

Fetching data from: https://fantasy.premierleague.com/api/bootstrap-static/
Data saved to: raw_data/season_data_24_25.json
Saved


In [ ]:
#Collect Player Details
if static_data:
    all_player_ids = [player['id'] for player in static_data.get('elements', [])]
    print(f"Found {len(all_player_ids)} players")

    #players_to_collect = all_player_ids
    players_to_collect = all_player_ids[:50] #smaller sample to limit dwnld times. swap comments to dwnld full set.

    collect_count = 0
    for player_id in players_to_collect:
        filename = f"player_{player_id}_details.json"
        player_data = get_save_data(f"{data_url}element-summary/{player_id}/", filename, subdir = "player_details")
        if player_data:
            collect_count += 1
        time.sleep(0.1)
    print(f"\nCollected Details for {collect_count} players")
else:
    print("No data was collected")
    

Found 804 players
Fetching data from: https://fantasy.premierleague.com/api/element-summary/1/
Data saved to: raw_data/player_details/player_1_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/2/
Data saved to: raw_data/player_details/player_2_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/3/
Data saved to: raw_data/player_details/player_3_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/4/
Data saved to: raw_data/player_details/player_4_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/5/
Data saved to: raw_data/player_details/player_5_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/6/
Data saved to: raw_data/player_details/player_6_details.json
Fetching data from: https://fantasy.premierleague.com/api/element-summary/7/
Data saved to: raw_data/player_details/player_7_details.json
Fetching data fr